# ResNet-50: Adversarial Training

In [1]:
import os
import sys
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import seaborn as sns

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
tf.get_logger().setLevel("ERROR")

In [ ]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 300
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS = 10
INPUT_SHAPE=(224, 224, 3)

tf.random.set_seed(5)
dataset_dir = "../datasets"

# Change dataset_dir when run in google colab 
if 'google.colab' in sys.modules:
    from google.colab import drive

    drive.mount('/content/drive')
    dataset_dir = "/content/drive/Othercomputers/Big Mac/datasets"
    BATCH_SIZE = 430

physical_gpus = tf.config.list_physical_devices('GPU')
print("Using available GPUs: ", physical_gpus)

tf.keras.mixed_precision.set_global_policy('float32')

Using available GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Load ImageNet2012 dataset
def prepare_input_data(input):
    image = tf.cast(input['image'], tf.float32)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    label = input['label']
    return image, label

def make_dataset(ds):
    return (
        ds.map(prepare_input_data, num_parallel_calls=AUTOTUNE)
        .batch(BATCH_SIZE)
        .prefetch(AUTOTUNE)
    )


(train, validation, test), info = tfds.load(
    'imagenet2012_subset/10pct',
    split=['train', 'validation[:50%]', 'validation[50%:]'],
    shuffle_files=False,
    with_info=True,
    data_dir=dataset_dir
)

num_classes = info.features['label'].num_classes
class_names = info.features['label'].names

print(f"Train count: {info.splits['train'].num_examples}")
print(f"Validation count: {info.splits['validation[:50%]'].num_examples}")
print(f"Test count: {info.splits['validation[50%:]'].num_examples}")

train_dataset = make_dataset(train)
validation_dataset = make_dataset(validation)
test_dataset = make_dataset(test)

Train count: 128116
Validation count: 25000
Test count: 25000


In [4]:
# Load adversarial datasets

def _parse_image(input):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(input, feature_description)
    image_f16 = tf.io.parse_tensor(parsed_features['image'], out_type=tf.float16)
    label = parsed_features['label']
    image_f32 = tf.cast(image_f16, tf.float32)
    image_f32.set_shape([IMG_SIZE, IMG_SIZE, 3])
    return image_f32, label

def create_tf_dataset(file_paths):
    raw_dataset = tf.data.TFRecordDataset(file_paths, compression_type='GZIP')
    tf_dataset = raw_dataset.map(_parse_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return tf_dataset

# Get all adversarial datasets for train, validation, and testing
train_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenet2012_subset/train-*.tfrec')
validation_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenet2012_subset/validation-*.tfrec')
test_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenet2012_subset/test-*.tfrec')

print(f"Loaded {len(train_file_paths)} TFrecord train files")
print(f"Loaded {len(validation_file_paths)} TFrecord validation files")
print(f"Loaded {len(test_file_paths)} TFrecord test files")

# Create a TFRecordDataset
adv_train_dataset = create_tf_dataset(train_file_paths)
adv_validation_dataset = create_tf_dataset(validation_file_paths)
adv_test_dataset = create_tf_dataset(test_file_paths)

Loaded 298 TFrecord train files
Loaded 59 TFrecord validation files
Loaded 59 TFrecord test files


In [ ]:
# Merge Adversarial and Clean datasets for retraining
# buffer_size = 8

# merged_train_dataset = adv_train_dataset.concatenate(train_dataset)
# shuffled_train_dataset = merged_train_dataset.shuffle(buffer_size=buffer_size)

# merged_validation_dataset = adv_validation_dataset.concatenate(validation_dataset)
# shuffled_validation_dataset = merged_validation_dataset.shuffle(buffer_size=buffer_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2

print("Training ResNet-50 model...\n")

# Improved base model setup with proper regularization
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=INPUT_SHAPE,
    classes=1000
)
base_model.trainable = False

# Improve classification head with regularization and dropout
inputs = tf.keras.Input(shape=INPUT_SHAPE)
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)  # Add dropout for regularization
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = tf.keras.layers.BatchNormalization()(x)  # Add batch normalization
x = tf.keras.layers.Dropout(0.5)(x)  # Higher dropout before final layer
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

robust_model = tf.keras.Model(inputs, outputs)

# Improved optimizer with better learning rate schedule
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

robust_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Callbacks for better training control
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        f"{dataset_dir}/models/robust_resnet50_beta_1.keras",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Training new classification layers...")
robust_model.fit(
    adv_train_dataset,
    verbose=1,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=adv_validation_dataset,
    callbacks=callbacks
)



In [ ]:
print("Fine tuning with model with lower learning rate")

# Unfreeze only the last 10 layers
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Unfreeze last 10 layers
    layer.trainable = False

# Fine-tuning with very low learning rate
fine_tune_lr = 1e-5
lr_schedule_finetune = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=fine_tune_lr,
    decay_steps=500,
    alpha=0.0
)

robust_model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=lr_schedule_finetune,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7
    ),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Updated callbacks for fine-tuning
callbacks_finetune = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=4,
        min_lr=1e-8,
        verbose=1
    ),
    ModelCheckpoint(
        f"{dataset_dir}/models/robust_resnet50_beta_1.keras",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

robust_model.fit(
    adv_train_dataset,
    verbose=1,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=adv_validation_dataset,
    callbacks=callbacks_finetune
)

# Save final model to file
robust_model.save(f"{dataset_dir}/models/robust_resnet50v1.keras")
